# User management processing

In [1]:
# THIS PART IS NOT NEEDED IN A JOB CREATE A PACKAGE INSTEAD
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.path.dirname(os.path.abspath(''))))
sys.path.append(parent_dir)

In [2]:
from local_packages.awesome_spotify_packages.request import UserManager as user_class, AuthorizationManager as auth_class
from local_packages.awesome_spotify_packages.aws import SecretManager as secret_manager, S3Manager as s3_manager

In [3]:
# Secrets
client_token = secret_manager('dev/awesome_api_spotify/client_token')
client_credentials = secret_manager('dev/awesome_api_spotify/client_credentials')
bucket_secret = secret_manager('dev/awesome_api_spotify/bucket_name')
# Test Bucket
s3 = s3_manager(bucket_secret)

In [4]:
#Init the auth class
auth = auth_class(client_token, client_credentials)
#Init the user class
user = user_class(auth = auth)

In [5]:
def fetch_top(type:str, limit:int = 50):
    offset = 0
    while True:
        obj = user.get_top(type = type, time_range = 'long_term', offset = offset, limit = limit).json()
        s3.put_s3('User/top/{}/{}_{}.json'.format(type, type, obj['offset']), obj['items'])
        offset += limit
        if obj['next'] == None:
            break

In [6]:
# Fetch my top artist in long term & storing them for further processing
for t in ['artists', 'tracks']:
    fetch_top(t)